In [40]:
from segment_anything import SamPredictor, sam_model_registry
import sys
import os
from segment_anything import SamPredictor, sam_model_registry
from PIL import Image
import cv2

import numpy as np 

sys.path.append("../sam-hq")

import demo_hqsam_light

def run_hq(point, image_dir, output_file):
    sam = sam_model_registry["vit_tiny"](checkpoint="../pretrained_checkpoint/sam_hq_vit_tiny.pth")

    device = "cuda"
    sam.to(device=device)
    sam.eval()
    predictor = SamPredictor(sam)


    image = cv2.imread(image_dir)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    predictor.set_image(image)
    # hq_token_only: False means use hq output to correct SAM output. 
    #                True means use hq output only. 
    #                Default: False
    hq_token_only = False 
    # To achieve best visualization effect, for images contain multiple objects (like typical coco images), we suggest to set hq_token_only=False
    # For images contain single object, we suggest to set hq_token_only = True
    # For quantiative evaluation on COCO/YTVOS/DAVIS/UVO/LVIS etc., we set hq_token_only = False

    # box prompt
    print(np.shape(image))
    input_point = np.array([point])
    input_label = np.ones(input_point.shape[0])

    masks, scores, logits = predictor.predict(
        point_coords=input_point,
        point_labels=input_label,
        box = None,
        multimask_output=False,
        hq_token_only=hq_token_only, 
    )
    result_path = './results/'
    os.makedirs(result_path, exist_ok=True)
    demo_hqsam_light.show_res(masks,scores,input_point, input_label, None, result_path + output_file, image)

In [41]:
run_hq([400, 600], "../squirrel.jpg", "squirrel")
run_hq([100, 100], "../legos.jpg", "lego")

c:\users\jmacy\websites\cs89.32\sam-hq\segment_anything\build_sam.py:93: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f, map_location=device)


<All keys matched successfully>
(824, 1100, 3)
Score: 0.984
<All keys matched successfully>
(238, 400, 3)
Score: 0.934
